In [21]:
import pandas as pd
import numpy as np

from pathlib import Path

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 120)

base_path = Path('/Users/brianrapanan/Desktop/6c571-project')
data_dir = base_path / 'data'

hitter_path = data_dir / 'Hitters_2023-24_Total.csv'
pitcher_path = data_dir / 'Pitchers_2015-2025_Total.csv'

hitters_df = pd.read_csv(hitter_path)
pitchers_df = pd.read_csv(pitcher_path)

hitters_df.head()


,mlbID,Age,#days,G,PA,AB,R,H,2B,3B,HR,RBI,BB,IBB,SO,HBP,SH,SF,GDP,SB,CS,BA,OBP,SLG,OPS,swings_competitive,percent_swings_competitive,contact,avg_bat_speed,hard_swing_rate,squared_up_per_bat_contact,squared_up_per_swing,blast_per_bat_contact,blast_per_swing,swing_length,swords,batter_run_value,whiffs,whiff_per_swing,batted_ball_events,batted_ball_event_per_swing,attempts,avg_hit_angle,anglesweetspotpercent,max_hit_speed,avg_hit_speed,ev50,fbld,gb,max_distance,avg_distance,avg_hr_distance,ev95plus,ev95percent,barrels,brl_percent,brl_pa,year,pa,bip,ba,est_ba,est_ba_minus_ba_diff,slg,est_slg,est_slg_minus_slg_diff,woba,est_woba,est_woba_minus_woba_diff,team_id,age,competitive_runs,bolts,hp_to_1b,sprint_speed,Season,WAR,Name,Games,WAR_per_162
0,408234,40,766,98,370,334,21,86,20,0,4,34,31,0,74,2,0,3,11,0,0,0.257,0.322,0.353,0.675,285.0,0.904762,225.0,68.187690,0.007018,0.386667,0.305263,0.142222,0.112281,7.389101,8.0,-7.821491,60.0,0.210526,124.0,0.435088,263.0,7.4,34.2,111.5,89.1,99.0,91.8,87.8,438.0,146.0,408.0,99.0,37.6,11.0,4.2,3.0,2023.0,370.0,263.0,0.257,0.247,0.010,0.353,0.363,-0.010,0.299,0.301,-0.002,116.0,40.0,127.0,0.0,5.09,23.1,TOTAL,-0.30,Miguel Cabrera,98,-0.495918
1,443558,42,856,49,152,143,9,35,5,1,5,23,6,1,46,2,0,1,2,1,0,0.245,0.283,0.399,0.681,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,135.0,42.0,39.0,0.0,4.91,24.7,TOTAL,-0.36,Nelson Cruz,49,-1.190204
2,444482,71,1146,232,701,649,84,170,38,1,16,87,42,1,128,4,0,6,12,6,1,0.527,0.625,0.806,1.431,787.0,1.799501,591.0,143.578267,0.436227,0.629017,0.473134,0.256653,0.193187,14.462935,21.0,-23.995537,196.0,0.494003,296.0,0.754810,328.0,8.2,31.1,111.5,89.3,100.8,92.8,87.2,415.0,151.0,403.0,145.0,44.2,16.0,4.9,3.8,2023.0,422.0,328.0,0.259,0.274,-0.015,0.381,0.400,-0.019,0.290,0.309,-0.019,254.0,71.0,306.0,0.0,9.04,53.9,TOTAL,1.57,David Peralta,232,1.096293
3,444489,36,850,4,12,12,1,3,0,0,1,1,0,0,3,0,0,0,1,0,0,0.250,0.250,0.500,0.750,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.0,0.0,0.0,0.0,0.00,0.0,TOTAL,0.00,Manny Pi\xc3\xb1a,4,0.000000
4,446334,37,735,90,291,259,28,55,12,0,11,32,26,0,94,1,1,4,9,0,0,0.212,0.283,0.386,0.669,111.0,0.902439,84.0,71.975303,0.198198,0.404762,0.306306,0.130952,0.099099,7.758970,2.0,-7.771029,27.0,0.243243,45.0,0.405405,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,0.000,109.0,37.0,74.0,0.0,4.76,26.2,TOTAL,0.42,Evan Longoria,90,0.756000


In [22]:
summary = pd.DataFrame([
    {'Dataset': 'Hitters', 'Rows': len(hitters_df), 'Columns': hitters_df.shape[1], 'WAR missing': hitters_df['WAR'].isna().sum()},
    {'Dataset': 'Pitchers', 'Rows': len(pitchers_df), 'Columns': pitchers_df.shape[1], 'WAR missing': pitchers_df['WAR'].isna().sum()},
])
summary


,Dataset,Rows,Columns,WAR missing
0,Hitters,848,80,0
1,Pitchers,2602,74,0


In [23]:
def prepare_numeric_features(df: pd.DataFrame, target_col: str = 'WAR') -> pd.DataFrame:
    numeric_df = df.copy()
    for col in numeric_df.columns:
        if col == target_col:
            continue
        if numeric_df[col].dtype == 'object':
            numeric_df[col] = pd.to_numeric(numeric_df[col], errors='coerce')
    numeric_df.replace([np.inf, -np.inf], np.nan, inplace=True)

    numeric_cols = numeric_df.select_dtypes(include=[np.number]).columns
    if target_col not in numeric_cols:
        numeric_df[target_col] = pd.to_numeric(numeric_df[target_col], errors='coerce')

    numeric_cols = numeric_df.select_dtypes(include=[np.number]).columns
    return numeric_df[numeric_cols].dropna(subset=[target_col])

hitters_numeric = prepare_numeric_features(hitters_df)
pitchers_numeric = prepare_numeric_features(pitchers_df)

hitters_numeric.describe().T.head(10)


,count,mean,std,min,25%,50%,75%,max
mlbID,848.0,648577.234670,54922.566119,408234.0,624427.00,665959.5,678885.00,810938.0
Age,848.0,60.501179,25.830297,20.0,31.75,65.5,81.00,120.0
#days,848.0,912.345519,432.359655,20.0,534.00,1176.0,1216.00,1590.0
G,848.0,181.699292,145.784732,4.0,48.00,145.5,306.25,493.0
PA,848.0,658.750000,606.444609,11.0,132.75,457.5,1079.50,2230.0
AB,848.0,589.410377,539.515320,11.0,118.75,410.5,966.25,1926.0
R,848.0,78.245283,79.780047,0.0,14.00,49.0,121.00,409.0
H,848.0,144.740566,141.889821,0.0,24.00,93.0,231.25,594.0
2B,848.0,28.397406,28.387999,0.0,5.00,18.0,46.00,138.0
3B,848.0,2.431604,3.821587,0.0,0.00,1.0,3.00,42.0


In [24]:
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import mutual_info_regression

rng = np.random.default_rng(42)


In [25]:
def top_correlations(df: pd.DataFrame, target_col: str = 'WAR', n: int = 10, method: str = 'pearson') -> pd.Series:
    feature_cols = [c for c in df.columns if c != target_col]
    corr = df[feature_cols].corrwith(df[target_col], method=method).dropna()
    return corr.abs().sort_values(ascending=False).head(n)


def top_mutual_info(df: pd.DataFrame, target_col: str = 'WAR', n: int = 10) -> pd.Series:
    feature_cols = [c for c in df.columns if c != target_col]
    X = df[feature_cols]
    y = df[target_col]

    valid_mask = y.notna()
    X = X.loc[valid_mask]
    y = y.loc[valid_mask]

    imputer = SimpleImputer(strategy='median', keep_empty_features=True)
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=feature_cols, index=X.index)

    mi = mutual_info_regression(X_imputed, y, random_state=42)
    return pd.Series(mi, index=feature_cols).sort_values(ascending=False).head(n)


def feature_importance_random_forest(df: pd.DataFrame, target_col: str = 'WAR', n: int = 10) -> pd.Series:
    feature_cols = [c for c in df.columns if c != target_col]
    X = df[feature_cols]
    y = df[target_col]

    valid_mask = y.notna()
    X = X.loc[valid_mask]
    y = y.loc[valid_mask]

    imputer = SimpleImputer(strategy='median', keep_empty_features=True)
    X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=feature_cols, index=X.index)

    model = RandomForestRegressor(n_estimators=400, random_state=42, n_jobs=-1)
    model.fit(X_imputed, y)

    importances = model.feature_importances_
    return pd.Series(importances, index=feature_cols).sort_values(ascending=False).head(n)



In [26]:
def summarize_feature_importance(df: pd.DataFrame, label: str, target_col: str = 'WAR') -> None:
    methods = {
        'Pearson |r|': top_correlations(df, target_col=target_col, method='pearson'),
        'Spearman |ρ|': top_correlations(df, target_col=target_col, method='spearman'),
        'Mutual Information': top_mutual_info(df, target_col=target_col),
        'Random Forest Importance': feature_importance_random_forest(df, target_col=target_col)
    }

    output_frames = []
    for method_name, series in methods.items():
        temp = series.reset_index()
        temp.columns = ['Feature', 'Score']
        temp['Rank'] = np.arange(1, len(temp) + 1)
        temp['Method'] = method_name
        output_frames.append(temp)

    summary = pd.concat(output_frames, ignore_index=True)
    summary = summary[['Method', 'Rank', 'Feature', 'Score']]
    display(f"Top features for {label}")
    display(summary)


summarize_feature_importance(hitters_numeric, label='Hitters')


'Top features for Hitters'

,Method,Rank,Feature,Score
0,Pearson |r|,1,R,0.814575
1,Pearson |r|,2,BB,0.786164
2,Pearson |r|,3,barrels,0.768132
3,Pearson |r|,4,RBI,0.766649
4,Pearson |r|,5,ev95plus,0.764040
5,Pearson |r|,6,HR,0.763823
6,Pearson |r|,7,H,0.757900
7,Pearson |r|,8,2B,0.755278
8,Pearson |r|,9,slg,0.753042
9,Pearson |r|,10,pa,0.753015


In [27]:
summarize_feature_importance(pitchers_numeric, label='Pitchers')


'Top features for Pitchers'

,Method,Rank,Feature,Score
0,Pearson |r|,1,SO,0.854837
1,Pearson |r|,2,W,0.847427
2,Pearson |r|,3,IP,0.817513
3,Pearson |r|,4,Pit,0.807893
4,Pearson |r|,5,AB,0.807823
5,Pearson |r|,6,BF,0.803673
6,Pearson |r|,7,pa,0.802909
7,Pearson |r|,8,bip,0.781170
8,Pearson |r|,9,attempts,0.781170
9,Pearson |r|,10,ev95plus,0.765833
